In [2]:
import subprocess
import os
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd
import re
import datetime

In [3]:
file = "/home/jupyter/gitlab/2023-pols-na-dna-spring-cleaning/files/delete.csv"
logfile = "/home/jupyter/gitlab/2023-pols-na-dna-spring-cleaning/logs/delete.log"

In [4]:
df = pd.read_csv(file)
#df

In [ ]:
client = bigquery.Client()

In [36]:
def check_or_create_dataset(client, dataset_id):
    try:
        client.get_dataset(dataset_id)
        print(f"{dataset_id} already exists.")
    except:
        print(f"Creating dataset: {dataset_id}")
        dataset = bigquery.Dataset(f'polsbigquery.{dataset_id}')
        job = client.create_dataset(dataset_id)
    
def copy_table(dataset_id, table_id):
    
    try:
        client.get_table(table_id)
        print(f"Copying: {table_id}.")
        job = client.copy_table(f'edwa-global.{dataset_id}.{table_id}', f'polsbigquery.{dataset_id}.{table_id}')
        job.result()
    except:
        print(f"Table {table_id} does not exists.")

def delete_table(client, dataset_id, table_id):
    try:
        client.get_table(table_id)
        print(f"Deleting: {table_id}")
        job = client.delete_table(f'edwa-global.{dataset_id}.{table_id}')
        job.result()
    except:
        print(f"Table {table_id} does not exists.")


In [32]:
# Hard code the schema column name later.
c = 'schema'
l = list(df[c].unique())
dataset_id = 'skunkworks'

In [35]:
for dataset_id in l:
    check_or_create_dataset(client, dataset_id)
    
    tables = df.loc[df[c] == dataset_id,'table'].to_list()
    for table_id in tables:
        copy_table(dataset_id, table_id)
        delete_table(dataset_id, table_id)
        

skunkworks already exists.
Table test_table_1 does not exists.
Table test_table_2 does not exists.
Table test_table_3 does not exists.
